In [1]:
import os
from pathlib import Path

# 1. MOVE TO ROOT
if os.getcwd().endswith("research"):
    os.chdir("..")
print(f"Current Directory: {os.getcwd()}")

Current Directory: d:\OneDrive\Desktop\disaster_victim_detection


In [2]:
from dataclasses import dataclass
from pathlib import Path
from src.victimDetector.constants import *
from src.victimDetector.utils.common import read_yaml, create_directories

In [5]:
# 2. DEFINE CONFIG ENTITY
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_imgsz: int
    params_model_type: str

In [6]:
# 3. CONFIG MANAGER
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params.yolo_params
        
        # Pointing to the Unzipped Data from Data Ingestion
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "C2A_Dataset")
        
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            training_data=Path(training_data),
            params_epochs=params.epochs,
            params_batch_size=params.batch_size,
            params_imgsz=params.imgsz,
            params_model_type=params.model_type
        )

        return training_config

In [7]:
# 4. COMPONENT
from ultralytics import YOLO
import yaml

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def create_data_yaml(self):
        """
        YOLO requires a data.yaml file with ABSOLUTE paths.
        We generate this dynamically to avoid path errors.
        """
        data_path = self.config.training_data
        
        # Define content dictionary
        yaml_content = {
            'path': str(data_path),
            'train': 'train/images',
            'val': 'val/images',
            'test': 'test/images',
            'names': {0: 'human'}
        }
        
        # Save to artifacts/training/data.yaml
        yaml_save_path = os.path.join(self.config.root_dir, "data.yaml")
        with open(yaml_save_path, 'w') as f:
            yaml.dump(yaml_content, f)
            
        return yaml_save_path

    def train(self):
        # 1. Create the data configuration file
        data_yaml_path = self.create_data_yaml()
        
        # 2. Load the base model (pretrained)
        # We load from the artifacts/prepare_base_model location
        model = YOLO(self.config.base_model_path)
        
        print(f"🚀 Starting YOLO Training for {self.config.params_epochs} epochs...")
        
        # 3. Train
        results = model.train(
            data=data_yaml_path,
            epochs=self.config.params_epochs,
            imgsz=self.config.params_imgsz,
            batch=self.config.params_batch_size,
            name="yolo_model",
            project=str(self.config.root_dir), # Save inside artifacts/training
            device="cpu" # Change to 0 if you have GPU setup
        )
        
        # 4. Save the BEST model to the specified path
        # YOLO saves usually in artifacts/training/yolo_model/weights/best.pt
        # We copy it to our final destination
        import shutil
        best_model_generated = os.path.join(self.config.root_dir, "yolo_model", "weights", "best.pt")
        
        if os.path.exists(best_model_generated):
            shutil.copy(best_model_generated, self.config.trained_model_path)
            print(f"✅ Best model saved to {self.config.trained_model_path}")
        else:
            print("⚠️ Could not find best.pt file to copy.")

In [8]:
# 5. PIPELINE EXECUTION
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.train()
except Exception as e:
    raise e

[2026-01-17 00:31:32,819: INFO: common: YAML file loaded successfully: config\config.yaml]
[2026-01-17 00:31:32,823: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-17 00:31:32,824: INFO: common: created directory at: artifacts]
[2026-01-17 00:31:32,826: INFO: common: created directory at: artifacts\training]
🚀 Starting YOLO Training for 50 epochs...
New https://pypi.org/project/ultralytics/8.4.4 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.252  Python-3.10.19 torch-2.9.1+cpu CPU (Intel Core i7-10750H 2.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=artifacts\training\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=Fa

KeyboardInterrupt: 